# Elasticsearch Retrieal with OpenAI

### Overview

In this notebook we will launch elastic search locally and then ask some questions related to 10-K filings of top 10 S&P500 companies

1. Set up enviornment
2. Build search tool to query our ES instance
3. Test the search tool
4. Create OpenAI client with access to the tool
5. Compare OpenAI's response with and without access to the tool

# Imports and start ElasticSearch

In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

from retriever.util import get_logger
from retriever.searcher.searchtools.elasticsearch import ElasticsearchSearchTool

logger = get_logger()

In [2]:
from retriever.searcher.constants import SEC_FILINGS_SEARCH_TOOL_DESCRIPTION
sec_search_tool = ElasticsearchSearchTool(SEC_FILINGS_SEARCH_TOOL_DESCRIPTION)

Starting Elasticsearch...
Elasticsearch is starting. Please wait a few moments before it becomes available.


In [3]:
aapl_earnings = sec_search_tool.search("AAPL earnings 2022", n_search_results_to_use=3)

2024-02-22 22:15:39,940 - elastic_transport.transport - INFO - POST http://localhost:9200/sec_filings/_search [status:200 duration:0.478s]
2024-02-22 22:15:39,940 - elastic_transport.transport - INFO - POST http://localhost:9200/sec_filings/_search [status:200 duration:0.478s]
2024-02-22 22:15:39,940 - elastic_transport.transport - INFO - POST http://localhost:9200/sec_filings/_search [status:200 duration:0.478s]


In [4]:
aapl_earnings

'\n<search_results>\n<item index="1">\n<page_content>\nSearchResult(content=\'{__fields_set__=[metadata, page_content], __private_attribute_values__={_lc_kwargs={metadata={source=edgar_docs/AAPL/10-K/2023-11-02.htm}, page_content=UNITED STATES SECURITIES AND EXCHANGE COMMISSION Washington, D.C. 20549 FORM\\n10-K (Mark One) ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE\\nSECURITIES EXCHANGE ACT OF 1934 For the fiscal year ended September 30, 2023\\nor TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES\\nEXCHANGE ACT OF 1934 For the transition period from to Commission File Number:\\n001-36743 Apple Inc. (Exact name of Registrant as specified in its charter)\\nCalifornia 94-2404110 (State or other jurisdiction of incorporation or\\norganization) (I.R.S. Employer Identification No.) One Apple Park Way\\nCupertino , California 95014 (Address of principal executive offices) (Zip\\nCode) (408) 996-1010 (Registrant’s telephone number, including area code)\\nSecurities 